# Check Basics

> Latest stats are from 2020, either because scimago stopped collecting data, or newer data is deemed unreliable regarding citations

In [6]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("scimago-journals.csv")

In [3]:
df.head().T

,0,1,2,3,4
sourceid,28773,19434,20315,29431,21100812243
rank,1,2,3,4,5
title,Ca-A Cancer Journal for Clinicians,MMWR Recommendations and Reports,Nature Reviews Molecular Cell Biology,Quarterly Journal of Economics,Nature Reviews Materials
type,journal,journal,journal,journal,journal
issn,"15424863, 00079235","10575987, 15458601","14710072, 14710080","00335533, 15314650",20588437
h_index,168,143,431,259,108
total_docs_2020,47,10,115,40,92
ref_per_doc,73.45,129.2,73.38,68.33,115.57
sjr_best_quartile,Q1,Q1,Q1,Q1,Q1
total_docs_3years,119,9,338,110,264


In [31]:
def get_cats(df):
    _cs = df["categories"].str.split("; ")
    idk = "sourceid"
    _df = pd.DataFrame(
        {"catbase": _cs.sum(), idk: np.repeat(df[idk].values, _cs.str.len())}
    )
    return pd.concat([_df, _df["catbase"].str.extract("(.*) \((.*)\)")], axis=1).assign(
        field=lambda df: np.where(df.loc[:, 0].isna(), df["catbase"], df.loc[:, 0])
    ).rename(columns={1: "Q"}).loc[:, ["field", "Q", idk]]

In [46]:
cat_base = get_cats(df.head(10000)).merge(df)

In [47]:
field_pivot = (
    cat_base.fillna("no Q")
    .pivot_table(index="field", columns="Q", values="sourceid", aggfunc="count")
    .fillna(0)
    .assign(s=lambda df: df.sum(axis=1))
    .sort_values("s", ascending=False)
)
field_pivot.head(15)

Q,Q1,Q2,Q3,Q4,miscellaneous,no Q,s
field,,,,,,,
Medicine (miscellaneous),599.0,600.0,98.0,0.0,0.0,0.0,1297.0
Education,318.0,177.0,0.0,0.0,0.0,6.0,501.0
Sociology and Political Science,320.0,35.0,0.0,0.0,0.0,2.0,357.0
"Ecology, Evolution, Behavior and Systematics",163.0,152.0,0.0,0.0,0.0,0.0,315.0
Computer Science Applications,162.0,115.0,0.0,0.0,0.0,36.0,313.0
Molecular Biology,99.0,99.0,98.0,14.0,0.0,0.0,310.0
Economics and Econometrics,165.0,134.0,0.0,0.0,0.0,2.0,301.0
Biochemistry,108.0,108.0,70.0,0.0,0.0,0.0,286.0
Psychiatry and Mental Health,133.0,134.0,13.0,0.0,0.0,0.0,280.0


In [95]:
def draw_table(df):
    return (
        df.pivot_table(index="field", columns="Q")
        .loc[field_pivot.head(15).index]
        .loc[:, lambda df: df.isna().mean() < 0.5]
        .style.background_gradient(axis=0)
    )

In [94]:
(
    cat_base.groupby(["field", "Q"])[["h_index", "journal_rating", "total_docs_3years"]]
    .mean()
    .pipe(draw_table)
)

In [88]:
def gini(s):
    vc = s.value_counts(normalize=True)
    diffs = np.abs(vc.values.reshape(-1, 1) - vc.values.reshape(1, -1))
    return diffs.sum() / (2 * vc.shape[0] ** 2 * vc.mean())

def top5(s):
    return s.value_counts(normalize=True).head(5).sum()

## Concentration metrics by field

In [96]:
cat_base.groupby(["field", "Q"])[["country", "publisher"]].agg([gini, top5]).pipe(draw_table)